In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [5]:
geolocator = Nominatim(user_agent="myGeocoder123")
location = geolocator.geocode("KR 33 17 27")
print(location.address)

Carrera 33, Las Américas, Comuna San José, Armenia, Quindío, 630003-341, Colombia


In [6]:
location

Location(Carrera 33, Las Américas, Comuna San José, Armenia, Quindío, 630003-341, Colombia, (4.5430139, -75.685216, 0.0))

In [7]:
location.latitude

4.5430139

In [2]:
# df = pd.read_csv('../initialData/Reto MINTIC/Ofertas/Ofertar.csv',sep=';',nrows=10)
df = pd.read_parquet('ofertas.parquet.gzip')  
df = df.iloc[:10,:]
df.head()

,Nombre Barrio,Codigo Barrio,Codigo Manzana,Codigo Predio,Codigo Construccion,Codigo Resto,Fecha De Cargue,Direccion,Codigo Oferta,Tipo Oferta,Tipo Inmueble,Fuente,Chip,Area Terreno,Area Construida,Vr Inicial Venta,Vr Final Venta,Vr Inicial Arriendo,Vr Final Arriendo
0,ESTACION CENTRAL,6204,17,15,0,0,2019-09-30T00:00:00Z,KR 33 17 27,"5,00E+05",VENTA,Bodega,CAMPO,AAA0251LMRJ,"2060,8",2306,"7,00E+09",6880300000,0,0
1,COOPERATIVA DE SUB-OFICIALES,6501,3,10,0,0,2019-09-30T00:00:00Z,KR 71F 12 02,500001,VENTA,Casa uso mixto,CAMPO,AAA0080RXPP,"100,1","211,3",0,0,0,0
2,ESTACION CENTRAL,6204,4,4,1,6011,2019-09-30T00:00:00Z,CL 14 30 17 AP 611,500002,ARRIENDO,Apartamento,CAMPO,AAA0164RCHY,"18,04","50,44",0,0,0,0
3,ARBORIZADORA ALTA,2568,10,14,0,0,2019-09-30T00:00:00Z,CL 74 SUR 38A 60,500003,ARRIENDO PARCIAL,Casa uso mixto,CAMPO,AAA0028RFAF,42,42,0,0,1200000,1178000
4,ARBORIZADORA ALTA,2568,10,12,0,0,2019-09-30T00:00:00Z,CL 74 SUR 38A 52,500004,VENTA,Casa uso mixto,CAMPO,AAA0028REYN,42,"130,2","1,50E+08",145335000,0,0


In [9]:
from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#2- - create location column
df['location'] = df['Direccion'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [10]:
len(df.index)-len(df.dropna().index)

6

In [11]:

map1 = folium.Map(
    location=[5,-74],
    tiles='cartodbpositron',
    zoom_start=7,
)
df.dropna().apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1